In [ ]:
import os

import glob
import copy
import numpy as np
import scipy
import astropy.table
import astropy.io.fits
import astropy.units

import matplotlib.pyplot as plt
import matplotlib as mpl


import lezargus

import pycpd

%matplotlib Qt

ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integr

In [ ]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/spectre_dispersion/", exist_ok=True)

# Slice Pattern File

In [ ]:
# Reading the base file, and saving the new file.
slice_pattern_filename = "./base/spectre_dispersion/spectre_slice_pattern.txt"
pattern_data = np.genfromtxt(slice_pattern_filename, comments="#").T

pattern_slice_index = pattern_data[0]
pattern_slice_x = pattern_data[1]
pattern_slice_y = pattern_data[2]

# The pattern values in the file are given in millimeters, we convert it to
# meters per convention.
pattern_slice_x = pattern_slice_x / 1000
pattern_slice_y = pattern_slice_y / 1000

pattern_index_column = astropy.table.Column(
    pattern_slice_index, name="index", unit="", dtype=int
)
pattern_x_column = astropy.table.Column(
    pattern_slice_x, name="x", unit="m", dtype=float
)
pattern_y_column = astropy.table.Column(
    pattern_slice_y, name="y", unit="m", dtype=float
)

table_columns = [pattern_index_column, pattern_x_column, pattern_y_column]
pattern_table = astropy.table.Table(table_columns)
# Too much precision makes unnessary large file sizes.
pattern_filename = f"./products/spectre_dispersion/spectre_slice_pattern.dat"
pattern_table.write(
    pattern_filename,
    format="ascii.mrt",
    formats={"index": "%4d"}
    | {keydex: "%.3e" for keydex in pattern_table.keys()},
    overwrite=True,
)
pattern_pairs = np.array([pattern_slice_x, pattern_slice_y]).T

# Disperstion Table File

In [ ]:
# Reading the template file which we will use to generate all of the
# other points.
template_dispersion_filename = (
    "./base/spectre_dispersion/spectre_slice_dispersion.txt"
)
template_dispersion_table = astropy.table.Table.read(
    template_dispersion_filename,
    format="ascii.basic",
    delimiter="|",
    comment="#",
)
# Convert to meters.
template_dispersion_table["Left X"] /= 1000
template_dispersion_table["Left Y"] /= 1000
template_dispersion_table["Center X"] /= 1000
template_dispersion_table["Center Y"] /= 1000
template_dispersion_table["Right X"] /= 1000
template_dispersion_table["Right Y"] /= 1000

In [4]:
def get_points_dispersion_table(
    channel: str, wavelength: float, location: str
) -> np.ndarray:
    """Get the points from the dispersion table.

    Parameters
    ----------
    channel : str
        The channel: VIS, NIR, MIR.
    wavelength : float
        The wavelength desired, but it must exist.
    location : str
        The location of the points within the slice, either "Left", "Center",
        or "Right".

    Returns
    -------
    points : ndarray
        The points.
    """
    # We filter the main table.
    filter_channel = template_dispersion_table["Channel"] == channel
    filter_wavelength = template_dispersion_table["Wavelength"] == wavelength
    filter_index = filter_channel & filter_wavelength
    filtered_table = template_dispersion_table[filter_index]

    point_x = np.array(filtered_table[f"{location} X"])
    point_y = np.array(filtered_table[f"{location} Y"])
    points = np.array([point_x, point_y]).T
    return points


def transform_pattern(pattern: np.ndarray, anchors: np.ndarray) -> np.ndarray:
    """Register/transform the provided point pattern to fit the anchor points.

    Parameters
    ----------
    pattern : ndarray
        The point pattern that we are registering. For most cases, this is the
        36 slice point pattern.
    anchors : ndarray
        The anchor points we are fitting the pattern to. For most cases, this
        is the wavelength dispersion slice points.

    Returns
    -------
    new_pattern : ndarray
        The point pattern after being transformed to fit the anchor points.
    """
    # The order of the pattern and anchors is important.
    transformation_mode = "affine"
    if transformation_mode == "rigid":
        registration = pycpd.RigidRegistration(X=anchors, Y=pattern)
        new_pattern, (_scale, _rotation, _translation) = registration.register()
    elif transformation_mode == "affine":
        registration = pycpd.AffineRegistration(X=anchors, Y=pattern)
        new_pattern, (_matrix, _translation) = registration.register()
    else:
        raise ValueError("Unknown transformation mode.")
    # As the registration above does the transformations for us, we can ignore
    # them.

    print(_matrix)
    print(_translation)
    return new_pattern


def channel_long_name(short_name: str):
    """Just, write out the channel name.

    Parameters
    ----------
    short_name : str
        The short name: VIS, NIR, MIR

    Returns
    -------
    long_name : str
        The long name.
    """
    channel_translation = {"VIS": "visible", "NIR": "nearir", "MIR": "midir"}
    return channel_translation[short_name]

In [9]:
# The unique combinations available for the dispersion file.
dispersion_combinations = (
    # Channel, wavelength
    ["VIS", 0.40],
    ["VIS", 0.50],
    ["VIS", 0.85],
    # ...
    ["NIR", 0.85],
    ["NIR", 1.25],
    ["NIR", 2.40],
    # ...
    ["MIR", 2.40],
    ["MIR", 3.20],
    ["MIR", 4.20],
)
# The anchor points do not provide any indication of height, so we calculate
# the height of the slice assuming that the height to width ratio is always the
# same. Height / Width.
height_width_ratio = 2 / 72


# Color map functionality for debugging plots.
def color_function(wavelength: float):
    # Parameters
    min_wave = 0.40
    max_wave = 4.20

    # We use energy as map.
    min_energy = 1 / max_wave
    max_energy = 1 / min_wave
    energy = 1 / wavelength

    color_map = copy.deepcopy(mpl.colormaps["jet_r"])
    normalize = mpl.colors.Normalize(
        vmin=min_energy, vmax=max_energy, clip=True
    )
    color = color_map(normalize(energy))
    return color

### Doing the work...

In [10]:
# Table rows.
full_dispersion_rows = []

plt.figure()
for channeldex, wavedex in dispersion_combinations:
    # Left
    left_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Left"
    )
    left_pattern = transform_pattern(pattern=pattern_pairs, anchors=left_anchor)
    # Center
    center_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Center"
    )
    center_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=center_anchor
    )
    # Right
    right_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Right"
    )
    right_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=right_anchor
    )

    # We use the width to figure out the top and bottom points based on the
    # ratio.
    left_x_points = left_pattern.T[0]
    right_x_points = right_pattern.T[0]
    widths = np.abs(right_x_points - left_x_points)
    heights = height_width_ratio * widths
    half_height = heights / 2

    # We assume the top and bottom points are directly offset in Y by the half
    # height from the center.
    top_pattern = np.array(
        [
            [xdex, ydex + heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )
    bot_pattern = np.array(
        [
            [xdex, ydex - heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )

    # Debug plot.
    if True:
        # The found pattern points, in reverse order of importance.
        plt.scatter(*bot_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*top_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*right_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*left_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(
            *center_pattern.T, marker=".", color=color_function(wavedex)
        )
        # And the anchors to compare it to.
        plt.scatter(*left_anchor.T, marker="2", color="pink")
        plt.scatter(*center_anchor.T, marker="*", color="pink")
        plt.scatter(*right_anchor.T, marker="2", color="pink")
        # Millimeter coordinate tick marks are easier.
        plt.show()

    for sliceindex, (cdex, tdex, bdex, ldex, rdex) in enumerate(
        zip(
            center_pattern,
            top_pattern,
            bot_pattern,
            left_pattern,
            right_pattern,
        )
    ):
        # Convert wavelength to meters.
        wavedex_m = wavedex * 1e-6

        row_dict = {
            "channel": channel_long_name(short_name=channeldex),
            "slice": sliceindex + 1,
            "wavelength": wavedex_m * astropy.units.Unit("m"),
            "center_x": cdex[0] * astropy.units.Unit("m"),
            "center_y": cdex[1] * astropy.units.Unit("m"),
            "top_x": tdex[0] * astropy.units.Unit("m"),
            "top_y": tdex[1] * astropy.units.Unit("m"),
            "bottom_x": bdex[0] * astropy.units.Unit("m"),
            "bottom_y": bdex[1] * astropy.units.Unit("m"),
            "left_x": ldex[0] * astropy.units.Unit("m"),
            "left_y": ldex[1] * astropy.units.Unit("m"),
            "right_x": rdex[0] * astropy.units.Unit("m"),
            "right_y": rdex[1] * astropy.units.Unit("m"),
        }
        full_dispersion_rows.append(row_dict)

plt.show()

[[8.33146293e-01 1.67502187e-04]
 [5.39487986e-04 8.20472564e-01]]
[-0.00052338  0.00077894]
[[8.33055831e-01 2.65031066e-04]
 [5.33465971e-04 8.20290142e-01]]
[1.08113947e-06 7.80887719e-04]
[[8.33154223e-01 6.97489674e-05]
 [1.71307723e-04 8.20289223e-01]]
[0.00052606 0.00077865]
[[ 8.33863687e-01 -1.37314268e-04]
 [ 5.42623994e-04  8.21024160e-01]]
[-0.00052222 -0.00254358]
[[8.33667191e-01 1.72280283e-04]
 [3.53866114e-04 8.21207210e-01]]
[ 4.16825657e-06 -2.54549166e-03]
[[8.33764720e-01 3.74567104e-04]
 [1.71444346e-04 8.21020692e-01]]
[ 0.00052611 -0.00254685]
[[ 8.34679749e-01 -3.46386835e-04]
 [ 5.37488118e-04  8.21757034e-01]]
[-0.00051769 -0.00664025]
[[8.34579004e-01 2.64110755e-04]
 [5.37658007e-04 8.21757171e-01]]
[ 4.52582210e-06 -6.64019429e-03]
[[8.34685900e-01 7.68925886e-04]
 [5.40657393e-04 8.21942147e-01]]
[ 0.00052563 -0.00663988]
[[ 1.00045522e+00 -1.23226109e-04]
 [-2.30704390e-02  1.01445997e+00]]
[-0.00038336 -0.00022654]
[[9.99917907e-01 1.76594955e-04]
 [3.4

In [ ]:
# Creating and saving the table.
full_dispersion_table = astropy.table.Table(rows=full_dispersion_rows)

# Types
table_type_formats = {
    "channel": str,
    "slice": "{:d}",
    "wavelength": "{: .3e}",
    "center_x": "{: .9f}",
    "center_y": "{: .9f}",
    "top_x": "{: .9f}",
    "top_y": "{: .9f}",
    "bottom_x": "{: .9f}",
    "bottom_y": "{: .9f}",
    "left_x": "{: .9f}",
    "left_y": "{: .9f}",
    "right_x": "{: .9f}",
    "right_y": "{: .9f}",
}

full_dispersion_filename = (
    f"./products/spectre_dispersion/spectre_slice_dispersion.dat"
)
full_dispersion_table.write(
    full_dispersion_filename,
    format="ascii.mrt",
    formats=table_type_formats,
    overwrite=True,
)

In [12]:
test_dispersion_table = astropy.table.Table.read(
    full_dispersion_filename,
    format="ascii.mrt",
)